In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lreg').getOrCreate()

In [6]:
# Reading data from the CSV file
data = spark.read.csv('Logistic_Regression/titanic.csv',inferSchema =True , header = True)

In [7]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [8]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [9]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [10]:
my_cols = data.select (['Survived',
 'Pclass','Sex',
 'Age','SibSp',
 'Parch','Fare','Embarked'])

In [11]:
# Dropping the missing datas
my_final_data = my_cols.na.drop()

In [12]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer

In [23]:
# Creating string indexer using sex column data
gender_indexer = StringIndexer(inputCol='Sex' , outputCol='SexIndex')
# Male Female
# 0    1
# Below OneHotEncoder(convert column datas into the vector format based on which of the kinds) value
# [1,0][0,1]
gender_encoder = OneHotEncoder (inputCol='SexIndex', outputCol='SexVec')

In [24]:
# Creating string indexer using sex column data
embark_indexer = StringIndexer(inputCol='Embarked' , outputCol='EmbarkedIndex')
# Male Female
# 0    1
# Below OneHotEncoder(convert column datas into the vector format based on which of the kinds) value
# [1,0][0,1]
embark_encoder = OneHotEncoder (inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

In [25]:
assembler = VectorAssembler(inputCols=['Survived',
 'Pclass','SexVec',
 'Age','SibSp',
 'Parch','Fare','EmbarkedVec'] ,outputCol='features')

In [26]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [27]:
reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [28]:
# Creating pipeline and giving input that everything we wants to do
pipeline = Pipeline (stages=[gender_indexer , gender_encoder ,embark_indexer , embark_encoder , assembler , reg_titanic])

In [29]:
train_data , test_data = my_final_data.randomSplit([0.7,0.3])

In [30]:
fit_model = pipeline.fit (train_data)

In [35]:
results = fit_model.transform(test_data)

In [36]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [37]:
my_eval = BinaryClassificationEvaluator (rawPredictionCol='prediction' , labelCol='Survived')

In [39]:
AUC = my_eval.evaluate(results)
AUC

1.0